<a href="https://colab.research.google.com/github/harryguiacorn/Trading-Analysis-Statistics/blob/main/Historical_data_analysis_pull_back_frequencies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://copilot.microsoft.com/chats/kon792goLB4QqgTX6qvPf

# 📊 SPY Pullback Dashboard Summary

This dashboard analyzes historical SPY price data to identify and quantify major pullbacks (≥10%), recovery durations, and post-recovery vulnerability. It includes:

- **Pullback Events**: Each time SPY dropped ≥10% from an all-time high
- **Recovery Tracking**: Time taken to reach a new all-time high after a pullback
- **Stability Analysis**: Duration a new high lasts before the next major drop
- **Drawdown Metrics**: Maximum drawdown % and duration from each ATH
- **Trend Indicators**: 50-day and 200-day moving averages for context

### 📁 Outputs
- CSV, Excel, JSON files for data export
- Interactive charts:
  - Pullback % over time
  - Price with moving averages
- HTML dashboard with download links and visual summaries

### 🔍 Use Cases
- Historical pattern recognition
- Risk and recovery modeling
- ETF comparison (extendable to QQQ, DIA, etc.)
- Integration into broader market dashboards or trading systems

### 🛠️ Next Steps
- Add multi-ETF support
- Schedule recurring updates
- Annotate key market events (e.g., COVID crash, 2008 crisis)
- Integrate with Copilot Pages for iterative insight generation


In [1]:
import yfinance as yf
import pandas as pd
import plotly.express as px

def run_spy_pullback_dashboard():
    spy = yf.Ticker("SPY")
    df = spy.history(period="max")[['Close']].reset_index()
    df['MA_50'] = df['Close'].rolling(window=50).mean()
    df['MA_200'] = df['Close'].rolling(window=200).mean()
    df['Peak'] = df['Close'].cummax()
    df['Drawdown'] = (df['Close'] - df['Peak']) / df['Peak']
    df['Drawdown %'] = df['Drawdown'] * 100

    events = []
    all_time_high = df['Close'].iloc[0]
    high_date = df['Date'].iloc[0]

    for i in range(1, len(df)):
        price = df['Close'].iloc[i]
        date = df['Date'].iloc[i]

        if price > all_time_high:
            all_time_high = price
            high_date = date

        drop_pct = (all_time_high - price) / all_time_high
        if drop_pct >= 0.10:
            pullback_date = date
            pullback_pct = round(drop_pct * 100, 2)

            recovery_date = None
            for j in range(i + 1, len(df)):
                future_price = df['Close'].iloc[j]
                future_date = df['Date'].iloc[j]
                if future_price > all_time_high:
                    recovery_date = future_date
                    break

            next_drop_date = None
            new_high = future_price if recovery_date else None
            for k in range(j + 1, len(df)):
                post_price = df['Close'].iloc[k]
                post_date = df['Date'].iloc[k]
                if new_high and (new_high - post_price) / new_high >= 0.10:
                    next_drop_date = post_date
                    break

            events.append({
                "ATH Date": high_date,
                "ATH Price": round(all_time_high, 2),
                "Pullback Date": pullback_date,
                "Pullback %": pullback_pct,
                "Recovery Date": recovery_date,
                "Recovery Duration (days)": (recovery_date - pullback_date).days if recovery_date else None,
                "Next Drop Date": next_drop_date,
                "Stability Duration (days)": (next_drop_date - recovery_date).days if recovery_date and next_drop_date else None,
                "Max Drawdown %": round(df.loc[i:j, 'Drawdown %'].min(), 2) if recovery_date else None,
                "Drawdown Duration (days)": (recovery_date - high_date).days if recovery_date else None
            })

            all_time_high = price
            high_date = date

    result_df = pd.DataFrame(events)[[
        "ATH Date", "ATH Price", "Pullback Date", "Pullback %",
        "Recovery Date", "Recovery Duration (days)",
        "Next Drop Date", "Stability Duration (days)",
        "Max Drawdown %", "Drawdown Duration (days)"
    ]]

    for col in result_df.columns:
        if pd.api.types.is_datetime64_any_dtype(result_df[col]):
            result_df[col] = result_df[col].dt.tz_localize(None)

    result_df.to_csv("spy_pullback_analysis.csv", index=False)
    result_df.to_excel("spy_pullback_analysis.xlsx", index=False)
    result_df.to_json("spy_pullback_analysis.json", orient="records", indent=2)

    fig1 = px.scatter(result_df, x="Pullback Date", y="Pullback %",
                      title="SPY Pullback Percentages Over Time",
                      labels={"Pullback %": "Pullback (%)"},
                      hover_data=["ATH Date", "Recovery Date"])
    fig1.write_html("spy_pullback_chart.html")

    fig2 = px.line(df, x="Date", y=["Close", "MA_50", "MA_200"],
                   title="SPY Price with 50-day and 200-day Moving Averages")
    fig2.write_html("spy_moving_averages_chart.html")

    html_table = result_df.to_html(index=False, table_id="spyTable")

    html_content = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <title>SPY Pullback Analysis</title>
        <!-- DataTables Core -->
        <link rel="stylesheet" href="https://cdn.datatables.net/1.13.6/css/jquery.dataTables.min.css">
        <script src="https://code.jquery.com/jquery-3.7.1.min.js"></script>
        <script src="https://cdn.datatables.net/1.13.6/js/jquery.dataTables.min.js"></script>

        <!-- Buttons Extension -->
        <link rel="stylesheet" href="https://cdn.datatables.net/buttons/2.4.1/css/buttons.dataTables.min.css">
        <script src="https://cdn.datatables.net/buttons/2.4.1/js/dataTables.buttons.min.js"></script>
        <script src="https://cdn.datatables.net/buttons/2.4.1/js/buttons.html5.min.js"></script>
        <script src="https://cdn.datatables.net/buttons/2.4.1/js/buttons.print.min.js"></script>
        <script src="https://cdnjs.cloudflare.com/ajax/libs/jszip/3.10.1/jszip.min.js"></script>
        <script src="https://cdnjs.cloudflare.com/ajax/libs/pdfmake/0.2.7/pdfmake.min.js"></script>
        <script src="https://cdnjs.cloudflare.com/ajax/libs/pdfmake/0.2.7/vfs_fonts.js"></script>

        <!-- Column Filters -->
        <script src="https://cdn.datatables.net/plug-ins/1.13.6/api/fnFilterOnReturn.js"></script>

        <style>
            body {{ font-family: Arial, sans-serif; margin: 40px; }}
            h1 {{ color: #2c3e50; }}
            table {{ border-collapse: collapse; width: 100%; }}
            th, td {{ border: 1px solid #ddd; padding: 8px; text-align: center; }}
            th {{ background-color: #f2f2f2; }}
            tr:hover {{ background-color: #f5f5f5; }}
            .downloads {{ margin-bottom: 20px; }}
            .downloads a {{ margin-right: 15px; text-decoration: none; color: #2980b9; }}
            .highlight {{ background-color: #ffe6e6 !important; }}
            tfoot input {{ width: 100%; padding: 3px; box-sizing: border-box; }}
        </style>
    </head>
    <body>
        <h1>SPY Pullback Analysis</h1>
        <div class="downloads">
            <a href="spy_pullback_analysis.csv" download>📄 Download CSV</a>
            <a href="spy_pullback_analysis.xlsx" download>📊 Download Excel</a>
            <a href="spy_pullback_analysis.json" download>🧾 Download JSON</a>
            <a href="spy_pullback_chart.html" target="_blank">📈 View Pullback Chart</a>
            <a href="spy_moving_averages_chart.html" target="_blank">📊 View Moving Averages</a>
        </div>
        <table id="spyTable" class="display" style="width:100%">
            <thead>{html_table.split('<thead>')[1].split('</thead>')[0]}</thead>
            <tfoot>{html_table.split('<thead>')[1].split('</thead>')[0]}</tfoot>
            <tbody>{html_table.split('<tbody>')[1].split('</tbody>')[0]}</tbody>
        </table>

        <script>
            $(document).ready(function() {{
                // Add column filters
                $('#spyTable tfoot th').each(function () {{
                    $(this).html('<input type="text" placeholder="Search" />');
                }});

                var table = $('#spyTable').DataTable({{
                    dom: 'Bfrtip',
                    buttons: ['copy', 'csv', 'excel', 'pdf', 'print'],
                    pageLength: 10,
                    lengthMenu: [10, 25, 50, 100, "All"],
                    initComplete: function () {{
                        this.api().columns().every(function () {{
                            var that = this;
                            $('input', this.footer()).on('keyup change clear', function () {{
                                if (that.search() !== this.value) {{
                                    that.search(this.value).draw();
                                }}
                            }});
                        }});
                    }}
                }});

                // Conditional formatting: highlight drawdowns > 15%
                table.rows().every(function () {{
                    var data = this.data();
                    var dd = parseFloat(data[8]); // Max Drawdown %
                    if (!isNaN(dd) && dd <= -15) {{
                        $(this.node()).addClass('highlight');
                    }}
                }});
            }});
        </script>
    </body>
    </html>
    """


    with open("spy_pullback_analysis.html", "w", encoding="utf-8") as f:
        f.write(html_content)

run_spy_pullback_dashboard()
